<a href="https://colab.research.google.com/github/idan424/AccuLine-noisy-ECG-classification-assignment/blob/main/Data_analysis_and_modeling_acculine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
PATH = '/content/drive/MyDrive/Colab Notebooks/AccuLine Assignment'

import os

os.chdir('/content/drive/MyDrive/Colab Notebooks/AccuLine Assignment')

In [8]:
os.listdir('DATA')


['X_train.npy', 'X_test.npy', 'y_test.npy', 'y_train.npy']